In [1]:
import sys
sys.path.append("../")

from document_search.ocr import DocumentReader


In [2]:
documents = [
    "../data/SP-51.13330.2011.pdf",
    "../data/SP-60.13330.2020.pdf"
]


In [3]:
doc_reader = DocumentReader()

processed_docs = []
for document_path in documents:
    processed_doc, errors = doc_reader.process_pdf(document_path)
    processed_docs.append(processed_doc)


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/Users/stasstaf/PycharmProjects/document-search/document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(
/Users/stasstaf/PycharmProjects/document-search/venv2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
for doc in processed_docs:
    storage.add_document(doc, pbar=True)


115it [00:14,  7.86it/s]                         
196it [00:27,  7.21it/s]                         


In [6]:
from document_search.RAG.retrievers import RAGRetriever


In [7]:
!ls

demo_ocr.ipynb          full_pipeline_rag.ipynb
full_pipeline.ipynb     system_prompt.txt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"

rag_retriever = RAGRetriever(embedder=embedder, storage=storage, path_to_prompt='../document_search/RAG/system_prompt.txt')

/Users/stasstaf/PycharmProjects/document-search/document_search/RAG/retrievers.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [9]:
answer = rag_retriever.retrieve_answer(query, k=1, context_length=1)

/Users/stasstaf/PycharmProjects/document-search/document_search/RAG/retrievers.py:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer = self.llm_chain.run(


In [10]:
print(answer)

**Ответ:**

В соответствии с приведёнными отрывками из документов, через помещения вентиляционного оборудования не допускается прокладывать трубопроводы с легковоспламеняющимися и горючими жидкостями и газами.

Также, согласно пункту 10.6, через помещения для вентиляционного оборудования запрещено прокладывать канализационные трубопроводы, за исключением ливневой канализации и водоотведения из вышележащих помещений. Также не допускаются канализационные трубопроводы с прочистками и ревизиями.

Необходимо отметить, что в представленных отрывках не содержится информации о конкретных типах трубопроводов, которые не допускается прокладывать через помещения вентиляционного оборудования. Однако, исходя из общих принципов безопасности и нормативных требований, можно предположить, что через такие помещения не рекомендуется прокладывать трубопроводы с опасными веществами, такими как легковоспламеняющиеся и горючие жидкости и газы, а также канализационные системы с прочистками и ревизиями.
